Hyperparameters will vary according to your Dataset, Its a example if your have 500 examples, 280 steps.

In [ ]:
N_train        = 500       
batch_size     = 1
grad_accum     = 8
num_epochs     = 4
steps_per_epoch= math.ceil(N_train / (batch_size * grad_accum))
total_steps    = num_epochs * steps_per_epoch
warmup_steps   = 10
learning_rate  = 2e-5

For more try this, approx 625 steps for 500 eaxmples and 5 epochs (if it overfits or overtrained then adjust, moniter the training loss 
) -

In [ ]:
import math
from trl import SFTTrainer
from transformers import TrainingArguments

N_train = 500 #adjust according to your dataset size

per_device_batch_size = 1
grad_accum            = 4  # reduced to allow for more gradient updates
num_epochs            = 5  # increased to let the model see the data more times
learning_rate         = 4e-5 # slightly higher LR can help on short training runs

steps_per_epoch = math.ceil(N_train / (per_device_batch_size * grad_accum))
max_steps       = num_epochs * steps_per_epoch

# hey update SFTTrainer arguments also
trainer_args = TrainingArguments(
    per_device_train_batch_size = per_device_batch_size,
    gradient_accumulation_steps = grad_accum,
    warmup_ratio                = 0.1, 
    max_steps                   = max_steps,
    learning_rate               = learning_rate,
    lr_scheduler_type           = "cosine",
    optim                       = "adamw_8bit",
    fp16                        = True, 
    logging_steps               = 10,
    save_strategy               = "steps",
    save_steps                  = steps_per_epoch, # Save at the end of each epoch
    output_dir                  = "outputs",
)

# and in the SFTTrainer call
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    args=trainer_args,
    packing=True, # for efficient training
    max_seq_length=2048,
)

print(f"Training for {num_epochs} epochs, which is {max_steps} total steps.")